In [67]:
import torch
import torch.nn as nn

In [68]:
class Residual(nn.Module):
    def __init__(self, fns):
        super().__init__()
        self.fns = nn.ModuleList(fns)

    def forward(self, x):
        y = x
        for fn in self.fns:
            x = fn(x)
        return x + y

In [69]:
class ResidualModule(nn.Module):
    def __init__(self, input_dim, hidden_dim, norm=nn.BatchNorm1d, drop_prob=0.1):
        super().__init__()
        self.modules = [nn.Linear(input_dim, hidden_dim),
            norm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(drop_prob),
            nn.Linear(hidden_dim, input_dim),
            norm(input_dim),
        ]
        self.layers = nn.ModuleList([
            Residual(self.modules),
            nn.ReLU()
        ])

    def forward(self, X):
        for module in self.layers:
            X = module(X)
        return X

In [70]:
class ResNet(nn.Module):
    def __init__(self, input_dim, hidden_dim=100, num_blocks=3, output_dim=2, norm=nn.BatchNorm1d, drop_prob=0.1):
        super().__init__()
        self.modules = [
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        ]
        for i in range(num_blocks):
            self.modules.append(ResidualModule(hidden_dim, hidden_dim // 2, norm, drop_prob))
        self.modules.append(nn.Linear(hidden_dim, output_dim))
        self.layers = nn.ModuleList(self.modules)

    def forward(self, X):
        for module in self.layers:
            X = module(X)
        return X

In [71]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

X = []
y = []
manipulated_path = './data/manipulated'
original_path = './data/original'
print("processing fake datasets")
for img_name in tqdm(os.listdir(manipulated_path)):
    X.append(np.array(Image.open(f'{manipulated_path}/{img_name}')))
    y.append([1, 0])
print("processing true datasets")
for img_name in tqdm(os.listdir(original_path)):
    X.append(np.array(Image.open(f'{original_path}/{img_name}')))
    y.append([0, 1])
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)

processing fake datasets


100%|██████████| 8000/8000 [00:33<00:00, 241.44it/s]


processing true datasets


100%|██████████| 4000/4000 [00:16<00:00, 243.59it/s]


(12000, 299, 299, 3)
(12000, 2)


In [72]:
np.random.seed(42)
indices = np.random.permutation(X.shape[0])
X_shuffled = []
y_shuffled = []
for i in range(X.shape[0]):
    X_shuffled.append(X[indices[i]])
    y_shuffled.append(y[indices[i]])
X_shuffled = np.array(X_shuffled, dtype=np.float32)
y_shuffled = np.array(y_shuffled, dtype=np.float32)
train_X = X_shuffled[:int(X.shape[0] * 0.8)]
train_y = y_shuffled[:int(X.shape[0] * 0.8)]
valid_X = X_shuffled[int(X.shape[0] * 0.8):int(X.shape[0] * 0.9)]
valid_y = y_shuffled[int(X.shape[0] * 0.8):int(X.shape[0] * 0.9)]
test_X = X_shuffled[int(X.shape[0] * 0.9):]
test_y = y_shuffled[int(X.shape[0] * 0.9):]
print(train_X.shape[0])
print(valid_X.shape[0])
print(test_X.shape[0])

9600
1200
1200


In [73]:
import math

def process_epoch(X, y, model, opt=None, batch_size=100):
    if opt:
        model.train()
    else:
        model.eval()

    loss_func = nn.CrossEntropyLoss()
    correct, loss_sum, n_step, n_samples = 0, 0, 0, 0

    iter = math.ceil(X.shape[0] / batch_size)
    for batch in tqdm(range(iter)):
        X_batch = X[batch * batch_size:min((batch + 1) * batch_size, X.shape[0])].reshape((-1, X[0].flatten().shape[0]))
        y_batch = y[batch * batch_size:min((batch + 1) * batch_size, X.shape[0])]
        if opt:
            opt.zero_grad()
        pred_y = model(torch.tensor(X_batch))
        loss = loss_func(pred_y, torch.tensor(y_batch))
        '''
        for i in range(5):
            print(pred_y.detach().numpy()[i])
            print(pred_y.detach().numpy()[i].argmax())
            print(y_batch[i])
            print(y_batch[i].argmax())
        temp = np.array(pred_y.detach().numpy().argmax(axis=1) == y_batch.argmax(axis=1), dtype=np.int64)
        print(temp[:5])
        '''
        correct += np.sum(np.array(pred_y.detach().numpy().argmax(axis=1) == y_batch.argmax(axis=1), dtype=np.int64))
        if opt:
            loss.backward()
            opt.step()
        loss_sum += loss.detach().numpy()
        n_step += 1
        n_samples += X_batch.shape[0]

    return 1 - correct / n_samples, loss_sum / n_step

In [74]:
def train(
        batch_size=100,
        epochs=10,
        optimizer=torch.optim.Adam,
        lr=0.001,
        weight_decay=0.001,
        hidden_dim=100,
        device=None):
    if (device != None):
        model = ResNet(X[0].flatten().shape[0], hidden_dim=hidden_dim).to(device)
    else:
        model = ResNet(X[0].flatten().shape[0], hidden_dim=hidden_dim)
    opt = optimizer(model.parameters(), lr=lr, weight_decay=weight_decay)

    for epoch in range(epochs):
        print(f"Epoch {epoch}: ")
        train_acc, train_loss = process_epoch(train_X, train_y, model, batch_size=batch_size, opt=opt)
        test_acc, test_loss = process_epoch(test_X, test_y, model, batch_size=batch_size)
        print(f"Train Accuracy: {round(train_acc, 3)} Test Accuracy: {round(test_acc, 3)}")

In [75]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train(device=device)

Epoch 0: 


100%|██████████| 12/12 [00:00<00:00, 13.76it/s]


Train Accuracy: 0.438 Test Accuracy: 0.675
Epoch 1: 


100%|██████████| 12/12 [00:00<00:00, 16.81it/s]


Train Accuracy: 0.427 Test Accuracy: 0.431
Epoch 2: 


...

Train Accuracy: 0.418 Test Accuracy: 0.36
Epoch 8: 


100%|██████████| 12/12 [00:00<00:00, 18.78it/s]


Train Accuracy: 0.406 Test Accuracy: 0.572
Epoch 9: 


100%|██████████| 12/12 [00:00<00:00, 17.42it/s]

Train Accuracy: 0.391 Test Accuracy: 0.372
